# Machine Learning Assignment (First Part)
##### Name: Matvey Makhnov<br> 
Task 1: Detection of inconsistencies in flower descriptions in online floristry and delivery platforms is essential for success, customer retention, and satisfaction. Many companies providing online floristry services are increasingly utilizing deep learning solutions to ensure that a flower image displayed on their platform matches the given description or category. <br> <br>To implement a flower classification convolutional neural network (CNN) trained on the Flowers102 dataset

In [ ]:
import os 
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# set a size for our batches 
train_batch = 32
val_batch = 32 
test_batch = 32

train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(), 
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

validation_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
]) 


data_path = os.path.join('.', 'Flowers_102_Dataset')

#----------------------------------------------------------------------------------------------------------

#dataset = datasets.Flowers102(root = data_path, transform=None, download=True)
# set a size for our sets (training, validation, test) 
#train_size = int(0.8 * len(dataset))
#val_size = int(0.1 * len(dataset))
#test_size = len(dataset) - (train_size + val_size)
#train_dataset, val_dataset, test_dataset =  random_split(dataset, [train_size,val_size,test_size])

#----------------------------------------------------------------------------------------------------------

# I want to apply 2 different transforms compositions 
# that is why I'll use next code 
train_dataset = datasets.Flowers102(root = data_path, split="train", transform=train_transform, download=True)
val_dataset = datasets.Flowers102(root = data_path, split="val", transform=validation_transform, download=True) 
test_dataset = datasets.Flowers102(root = data_path, split="test", transform=validation_transform, download=True)



train_loader = DataLoader(dataset=train_dataset, batch_size=train_batch,shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=val_batch,shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=test_batch, shuffle=False)

print(f"Using device: {device}")

Using device: cpu


The next step is to build our CNN architecture depends on Table 1 in  `F24.ML.Assignment.2.pdf` file <br> In this architecture I'll use only RELU activation function for all layes and for last one I'll apply softmax to get final results. In total we'll have 102 classes cause we have 102 types of flowers in our dataset

In [14]:
import torch.nn.functional as F 

class CNN_1(nn.Module):
    def __init__(self):
        super(CNN_1,self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1,padding=1)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=128 * 28 * 28, out_features=512)
        self.fc2 = nn.Linear(in_features=512,out_features=102)



    def forward(self,x):

        # the 1st convol layer input 224x224x3 output 114x112x32
        x = self.pool(self.conv1(x))
        # # the 2nd convol layer input 112x112x32 output 56x56x64
        x = self.pool(self.conv2(x))
        # the 3rd convol layer input 56x56x64 output 28x28x128
        x = self.pool(self.conv3(x))

        # here we have 28*28*128 values of feature map 

        # flattening 
        x = x.view(-1, 128 * 28 * 28)   


        # using weight matrics to 
        x = F.relu(self.fc1(x))
        x=self.fc2(x)

        return F.log_softmax(x, dim = 1)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_1 = CNN_1().to(device)

In [15]:
def counter_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [16]:
print(f"Number of parameters in our 1st CNN model: {counter_params(model_1)}")

Number of parameters in our 1st CNN model: 51526310


In [17]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

1020
1020
6149


Now I'll build training, validation and test function <br> We have to estimate our model on training, validation and test sets with using accuracy, loss and F1-score. I'll calculate average loss and accuracy for training set. And for validation and test sets I'll apply all of them (accuracy, average loss and definitely F1-score, cause it will give us ability to understand how to make our model better). <br> As a loss function I'll choose NLL Loss (Will explain latter why I choose it) 

In [31]:
from tqdm import tqdm
from sklearn.metrics import f1_score
from torch.utils.tensorboard import SummaryWriter

def train_1(model, device, loader, dataset, optimizer, epoch, writer):
    model.train()
    train_loss = 0 
    train_correct = 0 
    total = 0 
    
    for batch_idx, (images, labels) in enumerate(tqdm(loader, desc=f"Training epoch: {epoch}")):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        output = model(images)
        loss = F.nll_loss(output,labels)
        loss.backward()
        optimizer.step()

        train_loss +=loss.item()
        prediction = output.argmax(dim = 1, keepdim = True)
        #train_correct += (prediction == labels).sum().item()
        train_correct += prediction.eq(labels.view_as(prediction)).sum().item()
        total += labels.size(0)

    average_loss = train_loss/len(loader)
    accuracy = train_correct/len(loader.dataset) * 100.0 

    writer.add_scalar('Loss/Train', average_loss, epoch)
    writer.add_scalar('Accuracy/Train', accuracy, epoch)
    writer.flush()
    

    print(f"==> Epoch {epoch} Completed: Average loss: {average_loss:.6f}\tAccuracy: {accuracy:.3f}% ")


def validation_1(model, device, loader, dataset, epoch, writer):
    model.eval()
    validation_loss = 0
    val_correct = 0 
    v_labels_list = []
    v_prediction_list = []
    total = 0 

    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Valodation"):
            images, labels = images.to(device), labels.to(device)

            output = model(images)
            loss = F.nll_loss(output, labels)
            validation_loss +=loss.item()

            prediction = output.argmax(dim = 1, keepdim = True)
            #val_correct += (prediction == labels).sum().item()
            val_correct += prediction.eq(labels.view_as(prediction)).sum().item()

            v_labels_list.extend(labels.cpu().numpy())
            v_prediction_list.extend(prediction.cpu().numpy())
            total += labels.size(0)
            

    average_loss = validation_loss/len(loader)
    accuracy = val_correct/len(loader.dataset) * 100.0
    f1 = f1_score(v_labels_list, v_prediction_list, average="weighted")

    writer.add_scalar('Loss/Validation', average_loss, epoch)
    writer.add_scalar('Accuracy/Validation', accuracy, epoch)
    writer.add_scalar('F1-score/Validation', f1, epoch)
    writer.flush()

    print(f"==> Validation Completed: Average Loss: {average_loss:.6f}\tAccuracy: {accuracy:.2f}%\tF-1 Score: {f1:.4f}")

    # return accuracy and loss for tracking 
    return average_loss, accuracy



def test_1(model, device, loader,dataset):
    model.eval()
    test_loss = 0 
    test_correct = 0
    t_label_list = []
    t_prediction_list = []
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Test"):
            images, labels = images.to(device), labels.to(device)

            output = model(images)
            loss = F.nll_loss(output, labels)
            test_loss +=loss.item()

            prediction = output.argmax(dim = 1, keepdim = True)
            #test_correct += (prediction == labels).sum().item()
            test_correct += prediction.eq(labels.view_as(prediction)).sum().item()

            t_label_list.extend(labels.cpu().numpy())
            t_prediction_list.extend(prediction.cpu().numpy())
            total += labels.size(0)
            
    average_loss = test_loss/len(loader)
    accuracy = test_correct/len(loader.dataset) * 100
    f1 = f1_score(t_label_list,t_prediction_list,average="weighted")

    print(f"==>Test Completed: Avverage loss: {average_loss:.6f}\tAccuracy: {accuracy:.2f}%\tF-1 Score: {f1:.4f}")

    # return for tracking 
    return average_loss, accuracy

Now I'll train my model with applying SGD, learning rate = 0.001. Also I'll use TensorBoard <br> TensorBorad will help us to: <br>1. Inspect the model architecture <br>2. Create interactive of the visualization (...) <br> [Link for TensorBoard documentation](https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html)

In [32]:
from torch.utils.tensorboard import SummaryWriter
 
# before starting our training we have to 
# Initialize tensorboard writer
writer = SummaryWriter(log_dir="First_CNN_Model")

# our hyperparameters 
epochs = 10 
learning_rate = 0.001
momentum = 0.5

# Model and optimizer 
model = model_1.to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

best_accuracy = 0 

for epoch in range(1, epochs+1):
    print(f"\nEpoch {epoch}/{epochs}")
    
    # start our training
    train_1(model, device, train_loader,train_dataset, optimizer, epoch, writer)

    validation_1(model, device, val_loader, val_dataset, epoch, writer)

    test_loss, test_accuracy = test_1(model, device, test_loader, test_dataset)

    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        torch.save(model.state_dict(), "Best_in_the_1st_CNN.pt")
        print(f"The best model was saved with accuracy: {best_accuracy:.2f}%")


# in the end we have to use close method 
writer.close()



Epoch 1/10


Training epoch: 1:   0%|          | 0/32 [00:00<?, ?it/s]

Training epoch: 1: 100%|██████████| 32/32 [01:04<00:00,  2.00s/it]


==> Epoch 1 Completed: Average loss: 4.056027	Accuracy: 11.569% 


Valodation: 100%|██████████| 32/32 [00:25<00:00,  1.24it/s]


...

==> Epoch 10 Completed: Average loss: 2.157302	Accuracy: 44.804% 


Valodation: 100%|██████████| 32/32 [00:24<00:00,  1.28it/s]


==> Validation Completed: Average Loss: 3.577697	Accuracy: 19.02%	F-1 Score: 0.1626


Test: 100%|██████████| 193/193 [02:30<00:00,  1.28it/s]

==>Test Completed: Avverage loss: 3.830039	Accuracy: 14.82%	F-1 Score: 0.1293


I need to solve next problems: <br> 1. Figure out with accuracy why it's more than 100% <br> 2. Cnahge place where I'll put some writer cause in the end i just got point on my plot  (just wait untill pogram will not finished full)

**Conclusion:** 

Now I will create the 2nd model (optimize my 1st model with using next technic):<br> 1. Batch normalization <br> 2. Early stopping <br> 3. Dropout <br> 4. Scheduler for learning rate

In [ ]:
import os 
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms